# Model Project

In [1]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Model description

We consider a solow model for a closed economy with no growth in technology: 

* $K_t$ is capital
* $L_t$ is labor (growing with a constant rate of $n$)
* $Y_t = F(K_t,L_t)$ is GDP

We have the following equations:

$$Y_t = BK_t^\alpha L_t^{1-\alpha}$$
$$L_{t+1} = (1+n) L_t$$

**Saving** can be defined as the following fraction:

$$ S_t = sY_t,\,s\in(0,1) $$

These equations imply that the rate and the wage is the following:

$$r_t = \alpha B \big(\frac{K_t}{L_t}\big)^{\alpha-1}$$
$$w_t = (1-\alpha) B \big(\frac{K_t}{L_t}\big)^{\alpha}$$

Capital accumulation is therefore:

$$K_{t+1}= K_t + I_t-\delta K_t$$

$k_t$ can be defined as:

$$k_t = \frac{K_t}{Y_t}$$

Using the intertemporal budget constraint it is possible to show that the transition equation is:

$$ k_{t+1} = \frac{1}{1+n}(sBk_t^\alpha+(1-\delta)k_t)$$

# Steady state

## Analytical solution

In a standard solowmodel there is only a steady state in pr. capita values. In the transition equation we set $k_{t+1} = k_t = k$ and by doing that we get the steady state values for $y_t$ and $k_t$.
$$ k = \frac{1}{1+n}(sBk^\alpha+(1-\delta)k)$$

In [3]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
n = sm.symbols('n')
B = sm.symbols('B')

In [4]:
ss = sm.Eq(k,(s*B*k**alpha+(1-delta)*k)/((1+n)))

In [5]:
kss = sm.solve(ss,k)[0]
kss

(B*s/(delta + n))**(-1/(alpha - 1))

In [6]:
ss_func = sm.lambdify((s,B,n,alpha,delta),kss)

## Numerical solution

In [7]:
s = 0.2
B = 1
n = 0.01
alpha = 1/3
delta = 0.1

In [58]:
from scipy import optimize

def solve_for_ss(s,B,n,alpha,delta):
    """ solve for the steady state level of capital

    Args:
        s (float): saving rate
        n (float): population growth rate
        alpha (float): cobb-douglas parameter
        delta (float): capital depreciation rate 

    Returns:
        result (RootResults): the solution represented as a RootResults object

    """ 
    
    # a. define objective function
    f = lambda k: k**alpha
    obj_kss = lambda kss: kss - (s*B*f(kss) + (1-delta)*kss)/((1+n))

    #. b. call root finder
    result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='bisect')
    
    return result

In [60]:
solution = solve_for_ss(s,B,n,alpha,delta)
print(f' numerical solution is: {solution.root:.3f}')
print(f'analytical solution is: {ss_func(s,B,n,alpha,delta):.3f}')

 numerical solution is: 2.452
analytical solution is: 2.452


In [10]:
solve_for_ss(s,B,n,alpha,delta)

      converged: True
           flag: 'converged'
 function_calls: 48
     iterations: 46
           root: 2.4516358635037063

## Graphical representation

In [11]:
def transition(k,s,B,n,alpha, delta):
    """
    Returns:
    k1(float) capital in the next period
    
    Args:
    k (float): Initial stock of capital
    alpha (float): Capital income share.
    rho (float): Timepreference parameter
    n (float): Population growth
    
    output: 
    Computes the wage rate, optimal savings and capital in next period. 
    
    """
    # Raise ValueErrors
    if n>1:
        raise ValueError("Population growth cannot exceed 1")
    if alpha>1: 
        raise ValueError("Capital income share cannot exceed 1")

    k1 = (s*B*k**alpha+(1-delta)*k)/((1+n))
    return k1

In [12]:
def transition_curve(k,s,B,n,alpha, delta,T=1000,k_min=1e-20,k_max=6):
    """
    Returns:
    k_1(array): Array of capital in period 1
    k_2(array): Array of capital in period 2
    
    Args:
    alpha (float): Capital income share
    rho (float): Timepreference parameter
    n (float): Population growth
    T (int): Number of periods. Default 1000 periods.
    k_min (float): Minimum capital value. Default 1e-20.
    k_max (float): Maximum capital value. Default 6.
    
    output: 
    For every value of capital computes the value of capital in the next period using the transition equation. 
    """
    
    if n>1:
        raise ValueError("Population growth cannot exceed 1")
    if alpha>1: 
        raise ValueError("Capital income share cannot exceed 1")
    
    #grids
    k_1 = np.linspace(k_min,k_max,T)
    k_1 = np.linspace(1e-20,6,T)
    k_2 = np.empty(T)
    
    #construct array of "tomorrows" capital
    for i,k in enumerate(k_1):
        k_plus = transition(k,s,B,n,alpha, delta)
        k_2[i] = k_plus
        
    return k_1, k_2

In [13]:
def fig(s):
    """
    Returns:
    Plots the transition curve
    
    Args:
    rho(float): Timepreference parameter
    """
    #parameters
    alpha = 1/3
    s = s
    B = 1
    n = 0.2
    delta = 0.2
    #transition curve
    k_1, k_2 = transition_curve(k,s,B,n,alpha, delta,T=1000,k_min=1e-20,k_max=6)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(k_1,k_2, label="Transition curve") #transition curve
    ax.plot(k_1,k_1, '--', color='grey',label="45 degree") #45 degree line
    ax.set_xlabel('$k_t$')
    ax.set_ylabel('$k_t+1$')
    ax.set_title('Transition curve')
    ax.legend()
    ax.set_xlim([0,4])
    ax.set_ylim([0,4]);
    return

import ipywidgets as widgets
widgets.interact(fig, 
    s = widgets.FloatSlider(description='s', min=0, max=1, step=0.01, value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='s', max=1.0, step=0.01), Output()), _dom_classes=('w…

# Model extensions

We would like to specify the model so that it better fits Denmark. We do this by converting the model into a solowmodel on a small open economy. The new model is similar to the old model in a lot of ways, but some changes are made. In an open economy it is assumed that there is capital movements are free which means that the rate cannot differ between countries. When we assume that it is a small economy, we also assume that the economy cannot affect the global rental rate. 

The equation system that this new model is characterised by is the following:

$$ Y_t = BK_t^\alpha L_t^{1-\alpha}$$
$$L_{t+1} = (1+n) L_t$$
$$ S_t = sY_t,\,s\in(0,1) $$

In this model we have to define wealth which we name V and F is receivables:

$$ V_t = K_t + F_t $$

The reason for the creation of the wealth value is that we there is no steady state in capital pr. worker since capital no longer is accumelated. Instead we can examine the transition equation in wealth:

$$ v_{t+1} = \frac{sw*}{1+n}+\frac{1+s\bar{r}}{1+n}v_t $$

Where w and r are defined as in the beginning which means:
$$\bar{r} = \alpha B \big(\frac{K_t}{L_t}\big)^{\alpha-1}$$
$$w* = (1-\alpha) B \big(\frac{K_t}{L_t}\big)^{\alpha}$$

The equation for the rate ensures that:
$$ k_t = k* $$
Which means that the rate and the wage always is in steady state.

In [14]:
v = sm.symbols('v')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
n = sm.symbols('n')
B = sm.symbols('B')
k = sm.symbols('k')
w = sm.symbols('w')
r = sm.symbols('r')
ss = sm.Eq(v,(s*w/(1+n))+((1-delta+s*r)/(1+n))*v)

In [15]:
vss = sm.solve(ss,v)[0]
vss

s*w/(delta + n - r*s)

In [16]:
ssopen_func = sm.lambdify((s,B,n,r,k,w,alpha,delta),vss)

In [38]:
s = 0.2
B = 1
n = 0.01
k= 2
alpha = 1/3
delta = 0.1
w = (1-alpha)*B*k**alpha
r = alpha*B*k**(alpha-1)

In [36]:
from scipy import optimize

def solve_for_ssopen(s,B,n,k, alpha,delta):
    """ solve for the steady state level of capital

    Args:
        s (float): saving rate
        n (float): population growth rate
        alpha (float): cobb-douglas parameter
        delta (float): capital depreciation rate 

    Returns:
        result (RootResults): the solution represented as a RootResults object

    """ 
    
    # a. define objective function
    obj_vss = lambda vss: vss - ((s*w/(1+n))+((1-delta+s*r)/(1+n))*vss)

    #. b. call root finder
    result = optimize.root_scalar(obj_vss,bracket=[0.1,100],method='bisect')
    
    return result

In [39]:
solution = solve_for_ssopen(s,B,n,k,alpha,delta)
print(f' numerical solution is: {solution.root:.3f}')
print(f'analytical solution is: {ssopen_func(s,B,n,r,k,w,alpha,delta):.3f}')

 numerical solution is: 2.470
analytical solution is: 2.470
